<a href="https://www.kaggle.com/code/tanvikiran27/harmonic-invariant-basis?scriptVersionId=247503810" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import itertools as it
import sympy as sp 
import pandas as pd
from sympy import symbols, Function, diff, FiniteSet, simplify, re, AlgebraicNumber
from sympy import Q, ask
from sympy import *
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import nC
from sympy.matrices import diag, eye

import os
import zipfile
import shutil

for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/degeneracy-tables/4D.csv
/kaggle/input/degeneracy-tables/5D.csv
/kaggle/input/degeneracy-tables/2D.csv
/kaggle/input/degeneracy-tables/dataset-metadata.json
/kaggle/input/degeneracy-tables/3D.csv


In [2]:
# https://www.kaggle.com/code/nnjjpp/updating-a-dataset-with-a-notebook
from kaggle_secrets import UserSecretsClient
if os.path.exists('/root/.kaggle/'):
    pass
else:
    os.mkdir('/root/.kaggle/')
kaggle_API_key = UserSecretsClient().get_secret("key")

with open('/root/.kaggle/kaggle.json', 'w') as fid:
    fid.writelines(f'{{"username":"tanvikiran27","key":"{kaggle_API_key}"}}')

!chmod 600 /root/.kaggle/kaggle.json
print('Yay, it worked!')

Yay, it worked!


In [3]:
# https://www.kaggle.com/code/abdurrakibmollah/move-files-from-input-to-output-folder
dst_path = "/kaggle/working/"
for dirname, _, filenames in os.walk('/kaggle/input/degeneracy-tables'):
     for filename in filenames:
        src_path = os.path.join(dirname, filename)
        shutil.copy(src_path, dst_path)
        print('Copied '+filename)

Copied 4D.csv
Copied 5D.csv
Copied 2D.csv
Copied dataset-metadata.json
Copied 3D.csv


$n$ = dimension ($\mathbb{R}^n$ or $\mathbb{C}^n$) \
$k$ = real degree \
$(p,q)$ = complex bidegree

In [4]:
####### Change constants here #######
n = 3
k = 3
p = 2
q = 1
#####################################

# Setting up symbols
f = symbols('f', cls = Function)
x,y,z,w,zeta = symbols('x y z w zeta')
zs = symbols(f'z1:{n+1}', complex=True)
zbar = symbols(f'zbar1:{n+1}', complex=True)
ls = symbols(f'l1:{n+1}', int=True)

# newton potential (R^n) 
mag_real = 0
for i in range(n):
    mag_real += zs[i]**2 ## Euclidean norm squared
np_real = sp.sqrt(mag_real)**(2-n)

# newton potential (C^n)
mag_complex = 0
for i in range(n):
    mag_complex += zs[i]*zbar[i] ## Complex modulus squared
np_complex = sp.sqrt(mag_complex)**(2-2*n)

In [5]:
# Cyclic group (roots of unity) for lens spaces
m=5
zeta = exp(2*pi*I/m)
g = diag(zeta, zeta**(3))
G = FiniteSet()
for i in range(m):
    G += FiniteSet(g**i)
G

{Matrix([
[1, 0],
[0, 1]]), Matrix([
[exp(-4*I*pi/5),              0],
[             0, exp(-2*I*pi/5)]]), Matrix([
[exp(-2*I*pi/5),             0],
[             0, exp(4*I*pi/5)]]), Matrix([
[exp(2*I*pi/5),              0],
[            0, exp(-4*I*pi/5)]]), Matrix([
[exp(4*I*pi/5),             0],
[            0, exp(2*I*pi/5)]])}

In [6]:
# Multi index differentiation
def D(f,alpha,zs):
    df = f
    for i in range(n):
        df = diff(df,(zs[i],alpha[i]))
    return df

In [7]:
# Returns valid n-tuples
# https://www.geeksforgeeks.org/python-program-to-convert-dictionary-to-list-by-repeating-keys-corresponding-value-times/
def get_alpha_beta(deg):
    alphas = FiniteSet()
    for part in partitions(deg, m=n):
        part = list(it.chain.from_iterable(it.repeat(k, v) for k, v in part.items()))
        while len(part) < n:
            part.append(0)
        for perm in it.permutations(part):
            alphas += FiniteSet(perm)
    return alphas

In [8]:
# Turns rational functions in our basis into polynomials
def make_polynomial(f, mag):
    f = sp.numer(f).subs(mag, x)
    while f.is_polynomial(zs,zbar,x) == False:
        f = simplify(f * x)
        f = sp.numer(f).subs(mag, x)
    return expand(f.subs(x,abs(z)**2))

In [9]:
# For H_k(R^n) *NOT C^n
def real_dim():
    return nC(n+k-1,k) - nC(n+k-3,k-2)

In [10]:
# For H_k(C^n)
def dim_H_k():
    return nC(2*n+k-1,k) - nC(2*n+k-3,k-2)

In [11]:
# For H_p,q(C^n)
def dim_H_pq():
    return nC(n+p-1,p)*nC(n+q-1,q) - nC(n+p-2,p-1)*nC(n+q-2,q-1)

In [12]:
# Finding basis for H_k(R^n) - see Axler's Harmonic Function Theory p. 92
def real_basis():
    basis = FiniteSet()
    alphas = get_alpha_beta(k)
    for a in alphas:
        if a[0] > 1:
            alphas -= FiniteSet(a)
    print(f'# of elements = {len(alphas)}')
    print(f'Dimension from combinatorial expression = {real_dim()} \n')
    for a in alphas:
        d = D(np_real, a, zs)
        d = make_polynomial(d, mag_real)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [13]:
# Finding basis for H_p,q(C^n)
def basis_H_pq():
    basis = FiniteSet()
    alphas = get_alpha_beta(p)
    betas = get_alpha_beta(q)
    ab = FiniteSet()
    for a in alphas:
        for b in betas:
            if a[0]==0 or b[0]==0:
                ab += FiniteSet((a,b))
    print(f'# of elements = {len(ab)}')
    print(f'Dimension from combinatorial expression = {dim_H_pq()} \n')
    for (a,b) in ab:
        d = D(np_complex,b,zs)
        d = D(d,a,zbar)
        d = make_polynomial(d, mag_complex)
        print(d, '\n')
        basis += FiniteSet(d)
    return basis

In [14]:
# Regular complex Laplacian
def box(f):
    laplacian = 0
    for i in range(n):
        laplacian += diff(diff(f,zbar[i]),zs[i])
    return laplacian

In [15]:
# Kohn Laplacian as given in 2018 REU paper
def box_b(f):
    box_b = 0
    # first term
    for i in range(n): 
        box_b += -2*diff(diff(f,zbar[i]), zs[i])
        
    # second term
    for i in range(n): 
        box_b += 2*diff(zs[i]*q*f, zs[i])
        
    # third term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += 2*diff(zs[a]*zbar[i]*diff(f,zbar[i]), zs[a])
            
    # fourth term (double sum)
    for i in range(n): 
        for a in range(n):
            box_b += -2*diff(zs[a]*zbar[i]*zs[i]*q*f, zs[a])
    return box_b

In [16]:
# Just in case :)
def is_harmonic(f):
    return (box_b(f)/f).simplify().is_polynomial(mag_complex)

In [17]:
# Returns the Ikeda generating function for a given lens space
def F_lens(k, ls):
    F = 0
    zeta = exp(2*pi*I/k)
    for m in range(k):
        frac = 1-z*w
        for i in range(len(ls)):
            frac /= (z-zeta**(-m*ls[i]))*(w-zeta**(m*ls[i]))
        F += frac
    F = (1/k)*F
    return F

In [18]:
# General Ikeda generating function for any FINITE group
def F_gen(group):
    F = 0
    for g in group:
        n = sqrt(len(g))
        frac = (1-z*w) / ((z*eye(n)-g).det() * (w*eye(n)-g.C).det())
        F += frac
    F = (1/len(group))*F
    return F

In [19]:
# Determines if a matrix is unitary
def is_unitary(g):
    return g.inv() == g.H

# Helper function to check that the determinants aren't diverging
def get_abs(num):
    return sqrt(re(((num)*conjugate(num)).expand()))

# Generates a set of matrices that form the cyclic group generated by g
def make_group(g):
    G = FiniteSet(eye(sqrt(len(g))))
    gamma = g 
    i=1
    unitary = is_unitary(g)
    while gamma != eye(sqrt(len(g))):
        G += FiniteSet(gamma)
        dist = get_abs(det(gamma)-1) # To make sure the group isn't infinite
        # print(dist)
        gamma *= g
        if get_abs(det(gamma)-1) > dist and not unitary:
            print(f'{get_abs(det(gamma)-1)}>{dist}')
            raise Exception("This is an infinite group. Please try a different generator.")
        i += 1
    return G 

In [20]:
# Coefficients from Taylor series expansion for a two-variable function- read more at
# https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
def taylor_coeffs(f,deg):
    table = []
    for i in range(deg+1):
        row = []
        for j in range(deg+1-i):
            d = diff(f,w,j)
            d = diff(d,z,i)
            d = lambdify([z,w],d)
            term = re(d(0,0)/(factorial(i)*factorial(j))).round()
            row.append(term)
        while len(row) < deg+1:
            row.append('-')
        table.append(row)
    return table

In [21]:
# Gets a single degeneracy table (as a list of arrays) from the dataset
def clean_col(df,col):
    table = df[col].to_list()
    num_table = []
    while type(table[-1]) is float:
        table = table[:-1]
    for row in table:
        num_table.append(eval(row))
    return num_table

In [22]:
# Turns the order k and a list of l values into the name of a lens space (string)
# To avoid duplicates, the l values are sorted in ascending order (their order doesn't 
# affect the Ikeda generating function and thus doesn't affect the degeneracy tables)
def make_lens_str(k,ls):
    lens = "L("+str(k)+";"
    ls.sort()
    for l in ls:
        lens += str(l)+","
    lens = lens[:-1]
    lens += ")"
    return lens

In [23]:
# File lookup
def find_file(file):
    for dirname, _, filenames in os.walk('/kaggle/working'):
        if file in filenames:
            return True
    return False

In [24]:
# Update dataset
# https://www.kaggle.com/code/nnjjpp/updating-a-dataset-with-a-notebook
def update_data():
    with open('/kaggle/working/dataset-metadata.json', 'w') as json_fid:
        json_fid.write('{\n  "title": "Degeneracy Tables",\n  "id": "tanvikiran27/degeneracy-tables",\n  "licenses": [{"name": "CC0-1.0"}]}')
    !kaggle datasets version -p . -m "I hope this works"

In [25]:
# Renamed the files to get rid of the equal signs (it didn't help though...)
def rename_files():
    for dirname, _, filenames in os.walk('/kaggle/working'):
        for filename in filenames:
            if filename[-4:]=='.csv' and filename[:2]=='n=':
                copy = pd.read_csv(filename)
                file = filename[2] + 'D.csv'
                new = os.path.join(dirname, file)
                copy.to_csv(new, index = False)
                os.remove(os.path.join(dirname, filename))
                print(new)

In [26]:
# In case you need to clear the dataset for some reason
# CAUTION: This will delete the .csvs from the output directory and can potentially
# delete everything in the dataset- use with caution...
# https://www.kaggle.com/code/focusedmonk/delete-output-files-in-kaggle
def clear_csvs():
    import pathlib
    files_to_delete = './*.csv' # this considers only ".csv" files. If you want to delete all files, use "./*"
    files_list = pathlib.Path(os.getcwd()).glob(files_to_delete)
    for file_path in files_list:
        os.remove(file_path)

In [27]:
# Creates a degeneracy table for a lens space with all 0 <= p,q <= deg
# If there's already an entry for the lens space, this function finds it in the dataset
# Otherwise it adds the new table to the dataset
def lens_table(k, ls, deg):
    N = len(ls)
    lens = make_lens_str(k,ls)
    # file = 'n=' + str(N) + '.csv'
    file = str(N) + 'D.csv'
    path = "/kaggle/working/" + file
    if find_file(file):
        try:
            data = pd.read_csv(path)
        except pd.errors.EmptyDataError:
            data = pd.DataFrame()
    else:
        data = pd.DataFrame()
    if lens not in data or deg >= len(clean_col(data,lens)):
        f = F_lens(k,ls)
        T = taylor_coeffs(f,deg)
        if lens in data:
            data = data.drop(columns=[lens])
        new = pd.DataFrame({lens:T})
        added = data.merge(new, left_index = True, right_index = True, how = 'outer')
        added.to_csv(path, index = False)
        print("Created/updated " + file)
        update_data()
    else:
        T = clean_col(data,lens)
    return pd.DataFrame(T)

In [28]:
def degeneracy_table(generator, deg):
    N = sqrt(len(generator))
    name = str(generator)
    file = str(N) + 'D.csv'
    path = "/kaggle/working/" + file
    if find_file(file):
        try:
            data = pd.read_csv(path)
        except pd.errors.EmptyDataError:
            data = pd.DataFrame()
    else:
        data = pd.DataFrame()
    if name not in data or deg >= len(clean_col(data,name)):
        G = make_group(generator)
        f = F_gen(G)
        T = taylor_coeffs(f,deg)
        if name in data:
            data = data.drop(columns=[name])
        new = pd.DataFrame({name:T})
        added = data.merge(new, left_index = True, right_index = True, how = 'outer')
        added.to_csv(path, index = False)
        print("Created/updated " + file)
        update_data()
    else:
        T = clean_col(data,name)
    return pd.DataFrame(T)

In [29]:
M=Matrix([[0,I,0,0],
         [I,0,0,0],
         [0,0,0,I],
         [0,0,I,0]])
degeneracy_table(M,10)

Created/updated 4D.csv
Starting upload for file 5D.csv
100%|████████████████████████████████████████████| 186/186 [00:00<00:00, 368B/s]
Upload successful: 5D.csv (186B)
Starting upload for file 2D.csv
100%|██████████████████████████████████████| 1.86k/1.86k [00:00<00:00, 2.23kB/s]
Upload successful: 2D.csv (2KB)
Starting upload for file __notebook__.ipynb
100%|███████████████████████████████████████| 40.8k/40.8k [00:00<00:00, 117kB/s]
Upload successful: __notebook__.ipynb (41KB)
Starting upload for file 4D.csv
100%|██████████████████████████████████████| 1.09k/1.09k [00:00<00:00, 3.37kB/s]
Upload successful: 4D.csv (1KB)
Starting upload for file 3D.csv
100%|████████████████████████████████████████████| 290/290 [00:00<00:00, 852B/s]
Upload successful: 3D.csv (290B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5,6,7,8,9,10
0,1,0,4,0,19,0,40,0,85,0,140
1,0,7,0,36,0,93,0,200,0,355,-
2,4,0,44,0,132,0,312,0,580,-,-
3,0,36,0,148,0,388,0,776,-,-,-
4,19,0,132,0,417,0,904,-,-,-,-
5,0,93,0,388,0,951,-,-,-,-,-
6,40,0,312,0,904,-,-,-,-,-,-
7,0,200,0,776,-,-,-,-,-,-,-
8,85,0,580,-,-,-,-,-,-,-,-
9,0,355,-,-,-,-,-,-,-,-,-


In [30]:
K=Matrix([[1,0,0,0],
         [0,I,0,0],
         [0,0,-1,0],
         [0,0,0,-I]])
lens_table(5,[1,2,3,4],5)

Created/updated 4D.csv
Starting upload for file 5D.csv
100%|████████████████████████████████████████████| 186/186 [00:00<00:00, 564B/s]
Upload successful: 5D.csv (186B)
Starting upload for file 2D.csv
100%|██████████████████████████████████████| 1.86k/1.86k [00:00<00:00, 5.86kB/s]
Upload successful: 2D.csv (2KB)
Starting upload for file __notebook__.ipynb
100%|███████████████████████████████████████| 50.0k/50.0k [00:00<00:00, 135kB/s]
Upload successful: __notebook__.ipynb (50KB)
Starting upload for file 4D.csv
100%|██████████████████████████████████████| 1.27k/1.27k [00:00<00:00, 3.84kB/s]
Upload successful: 4D.csv (1KB)
Starting upload for file 3D.csv
100%|████████████████████████████████████████████| 290/290 [00:00<00:00, 886B/s]
Upload successful: 3D.csv (290B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5
0,1,0,2,4,7,12
1,0,3,8,14,24,-
2,2,8,16,32,-,-
3,4,14,32,-,-,-
4,7,24,-,-,-,-
5,12,-,-,-,-,-


In [31]:
D = Matrix([[Rational(1/2),sqrt(3)*I/2],
          [sqrt(3)*I/2,Rational(1/2)]])
A = diag(D,D,D)
degeneracy_table(A,10)

Created/updated 6D.csv
Starting upload for file 5D.csv
100%|████████████████████████████████████████████| 186/186 [00:00<00:00, 219B/s]
Upload successful: 5D.csv (186B)
Starting upload for file 6D.csv
100%|██████████████████████████████████████████| 791/791 [00:00<00:00, 2.27kB/s]
Upload successful: 6D.csv (791B)
Starting upload for file 2D.csv
100%|██████████████████████████████████████| 1.86k/1.86k [00:00<00:00, 5.67kB/s]
Upload successful: 2D.csv (2KB)
Starting upload for file __notebook__.ipynb
100%|███████████████████████████████████████| 55.9k/55.9k [00:00<00:00, 158kB/s]
Upload successful: __notebook__.ipynb (56KB)
Starting upload for file 4D.csv
100%|██████████████████████████████████████| 1.27k/1.27k [00:00<00:00, 3.15kB/s]
Upload successful: 4D.csv (1KB)
Starting upload for file 3D.csv
100%|████████████████████████████████████████████| 290/290 [00:00<00:00, 815B/s]
Upload successful: 3D.csv (290B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5,6,7,8,9,10
0,1,0,9,0,36,0,156,0,441,0,981
1,0,17,0,99,0,450,0,1464,0,3555,-
2,9,0,135,0,756,0,2754,0,7425,-,-
3,0,99,0,895,0,3852,0,11520,-,-,-
4,36,0,756,0,4298,0,14670,-,-,-,-
5,0,450,0,3852,0,15876,-,-,-,-,-
6,156,0,2754,0,14670,-,-,-,-,-,-
7,0,1464,0,11520,-,-,-,-,-,-,-
8,441,0,7425,-,-,-,-,-,-,-,-
9,0,3555,-,-,-,-,-,-,-,-,-


In [32]:
B=diag(1,D)
B
degeneracy_table(B,8)

Created/updated 3D.csv
Starting upload for file 5D.csv
100%|████████████████████████████████████████████| 186/186 [00:00<00:00, 561B/s]
Upload successful: 5D.csv (186B)
Starting upload for file 6D.csv
100%|██████████████████████████████████████████| 791/791 [00:00<00:00, 2.45kB/s]
Upload successful: 6D.csv (791B)
Starting upload for file 2D.csv
100%|██████████████████████████████████████| 1.86k/1.86k [00:00<00:00, 5.91kB/s]
Upload successful: 2D.csv (2KB)
Starting upload for file __notebook__.ipynb
100%|███████████████████████████████████████| 65.9k/65.9k [00:00<00:00, 168kB/s]
Upload successful: __notebook__.ipynb (66KB)
Starting upload for file 4D.csv
100%|██████████████████████████████████████| 1.27k/1.27k [00:00<00:00, 3.86kB/s]
Upload successful: 4D.csv (1KB)
Starting upload for file 3D.csv
100%|██████████████████████████████████████████| 717/717 [00:00<00:00, 2.00kB/s]
Upload successful: 3D.csv (717B)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5,6,7,8
0,1,1,2,2,3,3,6,6,9
1,1,2,3,4,5,8,11,14,-
2,2,3,5,6,10,13,19,-,-
3,2,4,6,10,14,20,-,-,-
4,3,5,10,14,21,-,-,-,-
5,3,8,13,20,-,-,-,-,-
6,6,11,19,-,-,-,-,-,-
7,6,14,-,-,-,-,-,-,-
8,9,-,-,-,-,-,-,-,-


In [33]:
B

Matrix([
[1,           0,           0],
[0,         1/2, sqrt(3)*I/2],
[0, sqrt(3)*I/2,         1/2]])

In [34]:
D2 = Matrix([[0,I],
            [I,0]])
B2 = diag(D2,1)
B2
degeneracy_table(B2,10)

Created/updated 3D.csv
Starting upload for file 5D.csv
100%|████████████████████████████████████████████| 186/186 [00:00<00:00, 548B/s]
Upload successful: 5D.csv (186B)
Starting upload for file 6D.csv
100%|██████████████████████████████████████████| 791/791 [00:00<00:00, 2.35kB/s]
Upload successful: 6D.csv (791B)
Starting upload for file 2D.csv
100%|██████████████████████████████████████| 1.86k/1.86k [00:00<00:00, 5.47kB/s]
Upload successful: 2D.csv (2KB)
Starting upload for file __notebook__.ipynb
100%|███████████████████████████████████████| 74.5k/74.5k [00:00<00:00, 175kB/s]
Upload successful: __notebook__.ipynb (75KB)
Starting upload for file 4D.csv
100%|██████████████████████████████████████| 1.27k/1.27k [00:00<00:00, 3.93kB/s]
Upload successful: 4D.csv (1KB)
Starting upload for file 3D.csv
100%|██████████████████████████████████████| 1.28k/1.28k [00:00<00:00, 3.96kB/s]
Upload successful: 3D.csv (1KB)
Dataset version creation error: Incompatible Dataset Type


,0,1,2,3,4,5,6,7,8,9,10
0,1,1,2,2,5,5,8,8,13,13,18
1,1,2,3,6,9,12,15,20,25,30,-
2,2,3,7,10,16,19,27,32,42,-,-
3,2,6,10,16,22,30,38,48,-,-,-
4,5,9,16,22,33,41,54,-,-,-,-
5,5,12,19,30,41,54,-,-,-,-,-
6,8,15,27,38,54,-,-,-,-,-,-
7,8,20,32,48,-,-,-,-,-,-,-
8,13,25,42,-,-,-,-,-,-,-,-
9,13,30,-,-,-,-,-,-,-,-,-


In [35]:
# Equivalent to Kohn Laplacian for n=2 and (we think) for higher dimensions
def R(f):
    R = 0
    for k in range(1,n):
        for j in range(k):
            R += M(j,k,M_bar(j,k,f))
    R *= -1
    return R

In [36]:
def M(j,k,f):
    return zbar[j]*diff(f,zs[k]) - zbar[k]*diff(f,zs[j])

In [37]:
def M_bar(j,k,f):
    return zs[j]*diff(f,zbar[k]) - zs[k]*diff(f,zbar[j])